In [15]:
import re
import pickle
import pandas as pd

In [16]:
with open('JobKorea_convert.pickle', 'rb') as f:
    df = pickle.load(f)

In [17]:
def get_content(rows):
    rowlist = [i for i in rows]
    Q = []
    content = []
    comment = []
    for row in rowlist:
        Q_ = row[0]
        content_ = row[1]
        comment_ = row[2]
        Q.append(Q_)
        content.append(content_)
        comment.append(comment_)
    
    return Q, content, comment

def preprocessing_applicatInfo(row):
    row = row.replace('\r','').replace('\n','')
    row = re.sub(' +', ' ', row)
    return row


def preprocessing_content(rows, type_='content'):
    contents = []
    for row in rows:
        if type_ == 'content':
            row = re.sub('아쉬운점 [0-9]','', row)
            row = re.sub('좋은점 [0-9]','', row)
            row = row[0:re.search('글자수', row).start()-1]
        try:
            row = row.replace('\r','').replace('\n','')
            row = re.sub(' +', ' ', row)
        except:
            pass
        contents.append(row)
    return contents


df['Question'] = df['paper'].map(get_content).map(lambda x:x[0])
df['content'] = df['paper'].map(get_content).map(lambda x:x[1])
df['comment'] = df['paper'].map(get_content).map(lambda x:x[2])
df = df.drop('paper', axis=1)

df['applicant_info'] = df['applicant_info'].map(preprocessing_applicatInfo)

df['content'] = df['content'].map(preprocessing_content)
df['comment'] = df['comment'].map(lambda x: preprocessing_content(x, type_='comment'))

In [117]:
# df.to_csv('JobKorea_preprocessd.csv', encoding='utf-8-sig')
# with open("JobKorea_preprocessd.pickle","wb") as fw:
#     pickle.dump(df, fw)

# drop 149
* 길이가 149이하인 답변은 삭제(질문, 전문가 조언 포함)

In [127]:
with open("JobKorea_preprocessd.pickle", 'rb') as f:
    df = pickle.load(f)

In [119]:
def makeup149(rows):
    AremainList = []
    BremainList = []
    CremainList = []
    for index, row in enumerate(rows['content']):
        if len(row) >= 149:
            AremainList.append(rows['Question'][index])
            BremainList.append(rows['content'][index])
            CremainList.append(rows['comment'][index])

    return pd.Series([AremainList,BremainList,CremainList], index=['Question','content','comment'])

In [120]:
df[['Question','content','comment']] = df[['Question','content','comment']].apply(makeup149, axis=1)

df = df[df['content'].apply(lambda x: True if x else False)]
# content가 빈 리스트인 row 제거

In [122]:
# df.to_csv('JobKorea_dropped.csv', encoding='utf-8-sig')
# with open("JobKorea_dropped.pickle","wb") as fw:
#     pickle.dump(df, fw)

# content 전처리

In [44]:
# with open("JobKorea_dropped.pickle", 'rb') as f:
#     df = pickle.load(f)

In [11]:
df = pd.read_csv('JobKorea_dropped(수기수정 및 중복제거 및 소제목제거).csv',
                encoding='cp949', index_col=0)

df = df.dropna(how='all')
df.index = df.index.astype(int)

df['Question'] = df['Question'].map(eval)
df['content'] = df['content'].map(eval)
df['comment'] = df['comment'].map(eval)

### 소제목 제거

In [12]:
def dropSubhead(df):
    content = df['content'].to_list()
    regex="\[.*\]|\s-\s.*" #[]
    regex1="\<.*\>|\s-\s.*"#<>
    regex2='\".*\"|\s-\s.*' #""
    regex3="\“.*\”|\s-\s.*" #“”
    regex4="\<<.*\>>|\s-\s.*" #<<>>
    regex5="▶▶.*◀◀|\s-\s.*" #▶▶ ◀◀
    regex6="◆.*◆|\s-\s.*" #◆ ◆

    for i,value in enumerate(content):
        for j, value2 in enumerate(value):
            text = re.sub(regex,'',"".join(value2)[0:100])
            text = re.sub(regex1,'',text)
            text = re.sub(regex2,'',text)
            text = re.sub(regex3,'',text)
            text = re.sub(regex4,'',text)
            text = re.sub(regex5,'',text)
            text = re.sub(regex6,'',text)
            text = text+value2[100:]
            text = re.sub('①','1.',text)
            text = re.sub('②','2.',text)
            text = re.sub('③','3.',text)
            text = re.sub('[\\#■]','',text) # \, # 제거
            content[i][j]= text
    df['content'] = content
    return df

df = dropSubhead(df)

### 영어로 작성된 자소서 drop

In [13]:
def isEng(rows):
    for row in rows:
        for i in row:
            if len(re.findall('[a-z]',i))/len(i) > 0.3:
                return True
            else: return False

df = df[~df[['content']].apply(isEng,axis=1)]

# Export

In [14]:
df.to_csv('JobKorea_processed.csv', encoding='utf-8-sig')
with open("JobKorea_processed.pickle","wb") as fw:
    pickle.dump(df, fw)